In [1]:

import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torch.utils.data import DataLoader
from torchvision.datasets import CocoDetection
from torchvision.transforms import functional as F
from torchmetrics.detection.mean_ap import MeanAveragePrecision
from tqdm import tqdm


c:\Users\a_fur\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def evaluate(model, data_loader, device, num_classes):
    model.eval()
    metric = MeanAveragePrecision()
 

    with torch.no_grad():
        for images, targets in tqdm(data_loader, desc="Evaluating"):
            images = [image.to(device) for image in images]
            predictions = model(images)
            
 
            for i, (target, pred) in enumerate(zip(targets, predictions)):
             
                true_boxes = []
                true_labels = []
                
                for annotation in target:
               
                    bbox = annotation['bbox']
             
                    bbox = [bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1] + bbox[3]]
                    true_boxes.append(bbox)
                    true_labels.append(annotation['category_id'])
                
                if not true_boxes:
                    continue
                
                true_boxes = torch.tensor(true_boxes, dtype=torch.float32).to(device)
                true_labels = torch.tensor(true_labels, dtype=torch.int64).to(device)
                
        
                pred_boxes = pred['boxes']
                pred_labels = pred['labels']
                pred_scores = pred['scores']
                
                
                keep = pred_scores > 0.3
                pred_boxes = pred_boxes[keep]
                pred_labels = pred_labels[keep]
                pred_scores = pred_scores[keep]
                
                if len(pred_boxes) == 0:
                    continue
          
                metric.update(
                    [dict(boxes=pred_boxes, scores=pred_scores, labels=pred_labels)],
                    [dict(boxes=true_boxes, labels=true_labels)]
                )


    map_score = metric.compute()
    


    return {
        "mAP_50": map_score['map_50'].item(),
        "mAP_75": map_score['map_75'].item()
    }

In [3]:
def get_model(num_classes):

    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    

    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    
    return model


num_classes = 9
model = get_model(num_classes)


model_path = "models/fasterrcnn_resnet50_epoch_10.pth"
model.load_state_dict(torch.load(model_path))
model.eval()

c:\Users\a_fur\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\a_fur\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\a_fur\AppData\Local\Temp\ipykernel_13876\558764029.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbit

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [4]:
class CocoTransform:
    def __call__(self, image, target):
        image = F.to_tensor(image)
        return image, target

In [5]:
def get_coco_dataset(img_dir, ann_file):
    return CocoDetection(
        root=img_dir,
        annFile=ann_file,
        transforms=CocoTransform()
    )


In [6]:
val_dataset = get_coco_dataset(
    img_dir="xray_knifes-6/test",
    ann_file="xray_knifes-6/test/_annotations.coco.json"
)

loading annotations into memory...
Done (t=0.04s)
creating index...
index created!


In [7]:
val_loader = DataLoader(val_dataset, batch_size=5, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

metrics = evaluate(model, val_loader, device, num_classes)

print("\nEvaluation Results:")
print(f"mAP@50: {metrics['mAP_50']:.4f}")
print(f"mAP@75: {metrics['mAP_75']:.4f}")

Evaluating: 100%|██████████| 258/258 [03:32<00:00,  1.21it/s]



Evaluation Results:
mAP@50: 0.8084
mAP@75: 0.3940
